# 5W

In [186]:
import activityinfo as AT
import pandas as pd
import numpy as np
import geopandas as gpd
import time

In [187]:
## definimos variables para el calculo mensual
mes_curso = 'AI2022-02.xlsx'
mes = '2022-01'

In [188]:
df_activity_info = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\DescargasMensuales\procesados\proc_'+mes_curso, index_col=0)

# Tablas de referencia

In [189]:
# Merge para appealing organization
Organizations_ref = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='ImplementingPartners')
Organizations_ref = Organizations_ref[['Nombre_ecu', 'Name']]
Organizations_ref = Organizations_ref[Organizations_ref['Nombre_ecu'].notna()]

In [190]:
# Sectores
sectores_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='sectores')

In [191]:
MarcoMonitoreo_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='marco_monitoreo')


In [192]:
delivery_mechanism = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='delivery_mech')

In [193]:
df_activity_info = df_activity_info[df_activity_info['Database'].notna()]
df_activity_info['Country'] = 'Ecuador'
df_activity_info['Country Admin1'] = df_activity_info['AInfo_Admi']
df_activity_info['Admin2'] = df_activity_info['AInfo_Ad_2']
df_activity_info = df_activity_info[df_activity_info['mes']==mes]

In [194]:
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='donante.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.drop(['Nombre_ecu'], axis=1, inplace=True)
df_activity_info.rename(columns = {'Name':'Appealing organisation Name'}, inplace = True)

In [195]:
# Implementation set up
df_activity_info['Implementation Set up'] = np.where(df_activity_info['implementación'] == 'Directa',
                                                 'No',
                                                 'Yes')

In [196]:
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='implementador.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.drop(['Nombre_ecu'], axis=1, inplace=True)
df_activity_info.rename(columns = {'Name':'Implementing partner Name'}, inplace = True)

In [197]:
# Month
df_activity_info['Month'] = df_activity_info['mes']


In [198]:
# Subsector
df_activity_info = pd.merge(df_activity_info, sectores_ref, how='left', left_on='sector',
                                    right_on='sector')

In [199]:
# Indicador
df_activity_info['Indicator'] = df_activity_info['indicador_eng']
df_activity_info = df_activity_info[df_activity_info['Indicator'].notna()]


In [200]:
#Activity
df_activity_info['Activity Name'] = df_activity_info['actividad']

In [201]:
#Activity Description
df_activity_info['Activity Description'] = ''

In [202]:
#COVID 19 Situation
df_activity_info['COVID 19 Situation'] = df_activity_info['¿esta_actividad_se_realiza_en_el_marco_de_respuesta_a_la_emergencia_sanitaria_por_el_covid-19?']
df_activity_info['COVID 19 Situation'] = np.where(df_activity_info['COVID 19 Situation'] == 'Si',
                                                 'Yes',
                                                 'No')


In [203]:
#RMRP Activity
df_activity_info['RMRP Activity'] = df_activity_info['¿esta_actividad_ha_sido_planificada_en_el_rmrp_2022?']
df_activity_info['RMRP Activity'] = np.where(df_activity_info['RMRP Activity'] == 'Si',
                                                 'Yes',
                                                 'No')

In [204]:
#CVA
df_activity_info['CVA'] = np.where(df_activity_info['CVA_tag'] == 'Si',
                                                 'Yes',
                                                 'No')


In [205]:
# Value (in USD)
USD_records = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Monto transferido'],
                                                       aggfunc='max',
                                                       fill_value=0,
                                                       margins='False')
df_activity_info = pd.merge(df_activity_info, USD_records, how='left', left_on='RecordId',
                                    right_on='RecordId')
df_activity_info.rename(columns = {'Monto transferido_y':'Value (in USD)'}, inplace = True)
df_activity_info['Value (in USD)'] = df_activity_info['Value (in USD)'].replace(np.nan, 0)
                                                

In [206]:
#Delivery mechanism

df_activity_info = pd.merge(df_activity_info, delivery_mechanism, how='left', left_on='mecanismo_de_entrega',
                                    right_on='mecanismo_de_entrega')
df_activity_info['Delivery mechanism'] = df_activity_info['Delivery mechanism'].replace(np.nan, '')

df_5w_part = df_activity_info[["RecordId","Country","Country Admin1","Admin2","Appealing organisation Name","Implementation Set up","Implementing partner Name","Month","Subsector","Indicator","Activity Name","Activity Description","COVID 19 Situation","RMRP Activity","CVA","Value (in USD)","Delivery mechanism"]]

df_5w_part.drop_duplicates(keep='first', inplace=True)

<ipython-input-206-e502d9900886>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5w_part.drop_duplicates(keep='first', inplace=True)


In [207]:
df_activity_info['aux']= df_activity_info['población meta'] + "-" + df_activity_info['frecuencia'] + "-" + df_activity_info['unidad_combinada']

In [208]:
df_activity_info['aux'] = df_activity_info['aux'].fillna('Quantity of output')
auxPivot = df_activity_info.pivot(columns=['aux'], values='Value').fillna(0)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)


In [209]:
#Quantity of output
Quantity_of_output = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Quantity of output'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, Quantity_of_output, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [210]:
# Total monthly beneficiaries
auxPivot=auxPivot[auxPivot.columns.drop(list(auxPivot.filter(regex='# de personas')))]
auxPivot=auxPivot[auxPivot.columns.drop('Quantity of output')]
auxPivot['Total monthly beneficiaries']=auxPivot.sum(axis = 1, skipna = True)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
total_monthly = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Total monthly beneficiaries'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, total_monthly, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [211]:
#New beneficiaries of the month
nuevos=list(auxPivot.filter(regex='Nuevo'))
auxPivot['New beneficiaries of the month']= auxPivot[nuevos].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
new_beneficiaries_month = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['New beneficiaries of the month'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, new_beneficiaries_month, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [212]:
#Refugees and Migrants IN DESTINATION
refugiados_migrantes=list(auxPivot.filter(regex='Refugiados y migrantes-Nuevo'))
auxPivot['Refugees and Migrants IN DESTINATION']= auxPivot[refugiados_migrantes].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
rrmm_in_destination = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Refugees and Migrants IN DESTINATION'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, rrmm_in_destination, how='left', left_on='RecordId',
                                    right_on='RecordId')


In [213]:
#Refugees and Migrants IN TRANSIT
df_5w_part['Refugees and Migrants IN TRANSIT'] = 0

In [214]:
#Host Communities Beneficiaries
comunidad_acogida=list(auxPivot.filter(regex='Comunidad de acogida-Nuevo'))
auxPivot['Host Communities Beneficiaries']= auxPivot[comunidad_acogida].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
ccaa_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Host Communities Beneficiaries'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, ccaa_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [215]:
print(comunidad_acogida)
df_5w_part.to_clipboard(excel=True)

['Comunidad de acogida-Nuevo-Hombres', 'Comunidad de acogida-Nuevo-Mujeres', 'Comunidad de acogida-Nuevo-Niñas', 'Comunidad de acogida-Nuevo-Niños', 'Comunidad de acogida-Nuevo-Otros']


In [216]:
#Refugees and Migrants PENDULARS
df_5w_part['Refugees and Migrants PENDULARS'] = 0
df_5w_part['Colombian Returnees'] = 0

In [217]:
# Women under 18
ninas=list(auxPivot.filter(regex='Nuevo-Niñas'))
auxPivot['Women under 18']= auxPivot[ninas].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
ninas_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Women under 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, ninas_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [218]:
# Men under 18
ninos=list(auxPivot.filter(regex='Nuevo-Niños'))
auxPivot['Men under 18']= auxPivot[ninos].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
ninos_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Men under 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, ninos_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [219]:
#Women above 18
mujeres=list(auxPivot.filter(regex='Nuevo-Mujeres'))
auxPivot['Women above 18']= auxPivot[mujeres].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
mujeres_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Women above 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, mujeres_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [220]:
#Men above 18
hombres=list(auxPivot.filter(regex='Nuevo-Hombres'))
auxPivot['Men above 18']= auxPivot[hombres].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
hombres_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Men above 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, hombres_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [221]:
# Other under 18
df_5w_part['Other under 18'] = 0

In [222]:
#Other above 18
otros=list(auxPivot.filter(regex='Nuevo-Otros'))
auxPivot['Other above 18']= auxPivot[otros].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
otros_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Other above 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, otros_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [223]:
#platform
df_5w_part["Platform"] = df_5w_part["Country"]

In [224]:
# final change appealing vs implementing

df_5w_part['Appealing organisation Name'] = np.where(df_5w_part['Implementation Set up'] == 'No',
                                                 df_5w_part['Implementing partner Name'],
                                                 df_5w_part['Appealing organisation Name'])

In [225]:
df_5w_part['Implementing partner Name'] = np.where(df_5w_part['Implementation Set up'] == 'No',
                                                 '',
                                                 df_5w_part['Implementing partner Name'])

In [231]:
# chequeo de valores invalidos
df_5w_part['rev'] = df_5w_part['Value (in USD)']+df_5w_part['Quantity of output']+df_5w_part['Total monthly beneficiaries']+df_5w_part['New beneficiaries of the month']+df_5w_part['Refugees and Migrants IN DESTINATION']+df_5w_part['Refugees and Migrants IN TRANSIT']+df_5w_part['Host Communities Beneficiaries']+df_5w_part['Refugees and Migrants PENDULARS']+df_5w_part['Colombian Returnees']+df_5w_part['Women under 18']+df_5w_part['Men under 18']+df_5w_part['Women above 18']+df_5w_part['Men above 18']+df_5w_part['Other under 18']+df_5w_part['Other above 18']
df_5w_part = df_5w_part[df_5w_part['rev'] > 0]

In [232]:
display(df_5w_part)

,RecordId,Country,Country Admin1,Admin2,Appealing organisation Name,Implementation Set up,Implementing partner Name,Month,Subsector,Indicator,...,Refugees and Migrants PENDULARS,Colombian Returnees,Women under 18,Men under 18,Women above 18,Men above 18,Other under 18,Other above 18,Platform,rev
0,c8b2w23kziszzb31n,Ecuador,Carchi,Tulcán,International Organization for Migration (IOM),No,,2022-01,Multipurpose Cash Assistance (MPC),# of individuals benefitting from multipurpose...,...,0,0,9,6,11,4,0,0,Ecuador,120.0
1,cp93cd9kzmsobe039,Ecuador,Carchi,Tulcán,HIAS,No,,2022-01,Multipurpose Cash Assistance (MPC),# of individuals benefitting from multipurpose...,...,0,0,5,2,3,3,0,0,Ecuador,512.0
2,caspryrkzhgyemu5q,Ecuador,Carchi,Tulcán,Norwegian Refugee Council (NRC),No,,2022-01,Multipurpose Cash Assistance (MPC),# of individuals benefitting from multipurpose...,...,0,0,0,0,25,21,0,0,Ecuador,1834.0
3,cwpz4jnkzhn6jxv5,Ecuador,Carchi,Tulcán,SOS Children's Villages,No,,2022-01,Multipurpose Cash Assistance (MPC),# of individuals benefitting from multipurpose...,...,0,0,2,1,0,0,0,0,Ecuador,612.0
4,cw9j2bskzl4rq8dl,Ecuador,Carchi,Tulcán,United Nations Children's Fund (UNICEF),Yes,HIAS,2022-01,Multipurpose Cash Assistance (MPC),# of individuals benefitting from multipurpose...,...,0,0,7,9,6,4,0,0,Ecuador,4681.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,cy5v9jdkzofc5jg2q,Ecuador,El Oro,Huaquillas,United Nations High Commissioner for Refugees ...,Yes,HIAS,2022-01,Food Security,"# of refugees, migrants and members of affecte...",...,0,0,16,18,21,18,0,0,Ecuador,292.0
805,cpd20k5kzk1cnhi5,Ecuador,El Oro,Huaquillas,United Nations High Commissioner for Refugees ...,No,,2022-01,Health,# of health facilities supported for the refug...,...,0,0,0,0,0,0,0,0,Ecuador,4.0
806,cyt1onmkzne13l99,Ecuador,El Oro,Huaquillas,International Committee for the Development of...,No,,2022-01,Health,# of refugees and migrants from Venezuela who ...,...,0,0,0,1,3,1,0,0,Ecuador,20.0
807,c4cxbizkzhzfw6kb,Ecuador,El Oro,Huaquillas,United Nations High Commissioner for Refugees ...,Yes,Permanent Human Rights defense Committee (CDH),2022-01,Health,# of refugees and migrants benefiting from pri...,...,0,0,7,9,21,7,0,0,Ecuador,176.0


In [226]:
df_5w_part.to_clipboard(excel=True)